# ASSOCIATION RULES

Oleh : Fitria Nur Aida 

In [1]:
import pandas as pd
import numpy as np

In [2]:
#import dataset
data=pd.read_csv("online_retail.csv")                                                                                                                                                                                                                                                                                                                                                                                                                   

In [3]:
data.head(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/10 08:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/10 08:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/10 08:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/10 08:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/10 08:26,3.39,17850.0,United Kingdom


In [4]:
data.shape

(541909, 8)

## PreProcessing

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
InvoiceNo      541909 non-null object
StockCode      541909 non-null object
Description    540455 non-null object
Quantity       541909 non-null int64
InvoiceDate    541909 non-null object
UnitPrice      541909 non-null float64
CustomerID     406829 non-null float64
Country        541909 non-null object
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [6]:
# Deteksi Missing Value
pd.DataFrame(np.sum(data.isnull()),columns=["Jumlah Missing"])

,Jumlah Missing
InvoiceNo,0
StockCode,0
Description,1454
Quantity,0
InvoiceDate,0
UnitPrice,0
CustomerID,135080
Country,0


> Missing value dalam variabel Description merupakan data produk yang tidak diketahui atau tidak tercatat nama produknya dalam suatu transaksi pembelian. Oleh karena itu dilakukan penghapusan (drop) pada data missing value tersebut. Sedangkan missing value pada variabel CustomerID merupakan data identitas pelanggan yang tidak diketahui ID pelanggannya. ID pelanggan yang tidak diketahui tidak mempengaruhi data transaksi pada variabel InvoiceNo dan Description sehingga missing value tersebut tidak diatasi.

In [7]:
# Penanganan Missing Value dengan Menghapus data missing
new=data.dropna(subset=["Description"])

In [8]:
pd.DataFrame(np.sum(new.isnull()),columns=["Jumlah Missing"])

,Jumlah Missing
InvoiceNo,0
StockCode,0
Description,0
Quantity,0
InvoiceDate,0
UnitPrice,0
CustomerID,133626
Country,0


> Variabel InvoiceNo terdapat data yang mengandung huruf C dan pada variabel Quantity terdapat nilai negatif yang menandakan adanya pembatalan transaksi, oleh karena itu dilakukan penghapusan data yang mengandung huruf C pada variabel InvoiceNo dan nilai negatif pada variabel Quantity.

In [10]:
#drop InvoiceNo yang mengandung "C" 
new['InvoiceNo'] = new['InvoiceNo'].astype('str')
new=new[~new['InvoiceNo'].str.contains('C')]
#drop InvoiceNo yang mengandung "Quantity" < 0
new=new[~new['Quantity']<0]

In [11]:
new.shape

(530693, 8)

In [12]:
new["InvoiceNo"]

0         536365
1         536365
2         536365
3         536365
4         536365
5         536365
6         536365
7         536366
8         536366
9         536367
10        536367
11        536367
12        536367
13        536367
14        536367
15        536367
16        536367
17        536367
18        536367
19        536367
20        536367
21        536368
22        536368
23        536368
24        536368
25        536369
26        536370
27        536370
28        536370
29        536370
           ...  
541879    581585
541880    581585
541881    581585
541882    581585
541883    581585
541884    581585
541885    581585
541886    581585
541887    581585
541888    581585
541889    581585
541890    581586
541891    581586
541892    581586
541893    581586
541894    581587
541895    581587
541896    581587
541897    581587
541898    581587
541899    581587
541900    581587
541901    581587
541902    581587
541903    581587
541904    581587
541905    581587
541906    5815

In [13]:
new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 530693 entries, 0 to 541908
Data columns (total 8 columns):
InvoiceNo      530693 non-null object
StockCode      530693 non-null object
Description    530693 non-null object
Quantity       530693 non-null int64
InvoiceDate    530693 non-null object
UnitPrice      530693 non-null float64
CustomerID     397924 non-null float64
Country        530693 non-null object
dtypes: float64(2), int64(1), object(5)
memory usage: 36.4+ MB


## Set Transactions Data

In [14]:
#Set Transactions
group_InvoiceNo=new.groupby('InvoiceNo')
transactions=[]
for name,group in group_InvoiceNo:
    transactions.append(list(group["Description"].map(str)))

## Market Basket Analysis dengan Association Rules Apriori

Install terlebih dahulu package "apyori"

In [15]:
from apyori import apriori

In [16]:
assrules=apriori(transactions,min_support=0.03, min_confidence=0.7, min_lift=1.1, min_length=2)
hasil=list(assrules)

In [17]:
# Visualising the result as a readeable dataframe
final_results = pd.DataFrame(np.random.randint(low=0, high=1, size=(len(hasil), 6)), columns=['GeneralRules', 'LeftRules', 'RightRules', 'Support', 'Confidence', 'Lift'])
index = 0
for g, s, i in hasil:
    final_results.iloc[index] = [' _&&_ '.join(list(g)), ' _&&_ '.join(list(i[0][0])), ' _&&_ '.join(list(i[0][1])), s, i[0][2], i[0][3]]
    index = index+1
# The most significant rules
final_results1 = final_results.sort_values("Confidence", ascending=0)
final_results1

,GeneralRules,LeftRules,RightRules,Support,Confidence,Lift
0,GREEN REGENCY TEACUP AND SAUCER _&&_ PINK REGE...,PINK REGENCY TEACUP AND SAUCER,GREEN REGENCY TEACUP AND SAUCER,0.031436,0.826371,16.393893
1,ROSES REGENCY TEACUP AND SAUCER _&&_ GREEN RE...,GREEN REGENCY TEACUP AND SAUCER,ROSES REGENCY TEACUP AND SAUCER,0.038141,0.756650,14.292598


In [18]:
count=1
for i, d in final_results1.head(10).iterrows():
    print('Rule #'+str(count)+':')
    print(d['LeftRules'])
    print('=> '+d['RightRules'])
    print('Support: '+str(d['Support'])+' - Confidence: '+str(d['Confidence'])+' - Lift: '+str(d['Lift']))
    print('--------------------')
    count=count+1

Rule #1:
PINK REGENCY TEACUP AND SAUCER
=> GREEN REGENCY TEACUP AND SAUCER
Support: 0.031436233611442196 - Confidence: 0.8263707571801567 - Lift: 16.393893169043977
--------------------
Rule #2:
GREEN REGENCY TEACUP AND SAUCER
=> ROSES REGENCY TEACUP AND SAUCER 
Support: 0.03814064362336114 - Confidence: 0.7566502463054187 - Lift: 14.29259789831699
--------------------


In [19]:
final_results1.to_excel(r'E:\2018_2019\SEMESTER 6\DATMIN\ETS DATMIN\hasil2.xlsx')

In [20]:
# Menghitung banyaknya transaksi
data["InvoiceNo"].value_counts()

573585     1114
581219      749
581492      731
580729      721
558475      705
579777      687
581217      676
537434      675
580730      662
538071      652
580367      650
580115      645
581439      635
580983      629
578344      622
538349      620
578347      606
537638      601
537237      597
536876      593
576617      593
536592      592
537823      591
576837      585
579508      578
577078      572
537240      568
577358      561
576618      552
576840      544
           ... 
579743        1
563597        1
558890        1
C537444       1
C545437       1
C542140       1
C562884       1
C542232       1
547536        1
561362        1
C571045       1
C565089       1
568404        1
579742        1
C550966       1
561638        1
573293        1
571249        1
555580        1
551240        1
538241        1
560651        1
576193        1
550979        1
C561017       1
C552765       1
C579992       1
544797        1
549494        1
560425        1
Name: InvoiceNo, Length:

> Probabilitas pembelian produk “PINK REGENCY TEACUP AND SAUCER” dan “GREEN REGENCY TEACUP AND SAUCER” secara bersamaan yaitu 0,031436 dari keseluruhan transaksi. Berdasarkan pembelian produk “PINK REGENCY TEACUP AND SAUCER”, sebesar 82,63% pelanggan juga membeli produk “GREEN REGENCY TEACUP AND SAUCER”. Nilai Lift lebih besar dari 1 sehingga pelanggan yang membeli produk “PINK REGENCY TEACUP AND SAUCER” memiliki kencerungan tinggi untuk membeli produk “GREEN REGENCY TEACUP AND SAUCER”.

> Probabilitas pelanggan membeli produk “GREEN REGENCY TEACUP AND SAUCER” dan “ROSES REGENCY TEACUP AND SAUCER” yaitu sebesar 0,038141 atau sebesar 3.8141% produk “GREEN REGENCY TEACUP AND SAUCER” dan “ROSES REGENCY TEACUP AND SAUCER” dibeli secara bersamaan dari keseluruhan transaksi. Persentase pelanggan membeli produk “ROSES REGENCY TEACUP AND SAUCER” sebesar 75,665% dari pembelian produk “GREEN REGENCY TEACUP AND SAUCER”. Sedangkan nilai Lift yang lebih besar dari 1 berarti pelanggan yang membeli produk “GREEN REGENCY TEACUP AND SAUCER” memiliki kecenderungan tinggi untuk membeli “ROSES REGENCY TEACUP AND SAUCER”.